## Visualization of Schism results

In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.tri as tri
from netCDF4 import Dataset
from scipy.io import loadmat
import pandas as pd
from scipy.interpolate import griddata
from tqdm import tqdm

In [ ]:
from mayavi import mlab

In [2]:
from pyPoseidon.utils import *


In [3]:
from matplotlib import animation,rc

rc('animation',html='html5')


In [4]:
PATH='/home/critechuser/SCHISM/EUROP/'

In [5]:
#g=loadmat('/home/critechproc/STORAGE/SCHISM/test1/nodes.mat')
#g['nodes'][:].shape

In [6]:
grid=PATH+'hgrid.gr3'

In [7]:
#extract number of elements, number of nodes
ne,nn = pd.read_csv(grid,header=0,skiprows=1,nrows=0,delim_whitespace=True)

In [8]:
ne=int(ne)
nn=int(nn)

In [9]:
#read lon,lat,depth for all nodes
q=pd.read_csv(grid,skiprows=2,header=None,delim_whitespace=True,engine='python',nrows=nn,names=['id','x','y','z'])

In [10]:
q=q.set_index(['id'])

In [11]:
q.head()

,x,y,z
id,,,
1,39.457138,47.262344,24.941161
2,39.457632,47.211644,24.919153
3,39.460017,47.150657,24.880700
4,39.389615,47.226132,25.379718
5,39.369948,47.271625,25.528124


In [12]:
e = pd.read_csv(grid,skiprows=nn+2,header=None,delim_whitespace=True,engine='python',nrows=ne,names=['id','nv','a','b','c'])

In [13]:
e=e.set_index(['id'])

### read output

In [20]:
r = Dataset(PATH+'outputs/schism_history.nc')

In [21]:
r.variables.keys()

[u'ocean_time',
 u'ocean_time_day',
 u'ocean_time_str',
 u'kbp00',
 u'idry',
 u'WATLEV',
 u'Uwind',
 u'Vwind',
 u'davx',
 u'davy',
 u'qnon']

In [ ]:
U = r.variables['davx'][:]
V = r.variables['davy'][:]

In [ ]:
h = r.variables['WATLEV'][:]

In [ ]:
h

In [22]:
time = r.variables['ocean_time_str'][:]

In [23]:
t = [''.join(x) for x in time] #join strings in readable form

In [24]:
len(t) # time steps

28

In [25]:
t

['20100202.000000',
 '20100203.000000',
 '20100204.000000',
 '20100205.000000',
 '20100206.000000',
 '20100207.000000',
 '20100208.000000',
 '20100209.000000',
 '20100210.000000',
 '20100211.000000',
 '20100212.000000',
 '20100213.000000',
 '20100214.000000',
 '20100215.000000',
 '20100216.000000',
 '20100217.000000',
 '20100218.000000',
 '20100219.000000',
 '20100220.000000',
 '20100221.000000',
 '20100222.000000',
 '20100223.000000',
 '20100224.000000',
 '20100225.000000',
 '20100226.000000',
 '20100227.000000',
 '20100228.000000',
 '20100301.000000']

## plot

In [ ]:
triangles = e.loc[:,['a','b','c']].values

In [ ]:
triangles

In [ ]:
triangles.max()

In [ ]:
tri3 = triangles-1 # adjust for python

In [ ]:
x = q.loc[:,['x']].values.flatten()
y = q.loc[:,['y']].values.flatten()

In [ ]:
x.shape,y.shape

In [ ]:
z = q.loc[:,['z']].values.flatten() #depth

In [ ]:
minlon=-35. # lat/lon window
maxlon=43.
minlat=24.5
maxlat=76.5

In [ ]:
plt.figure(figsize=(12,10))
plt.gca().set_aspect('equal')
plt.tricontourf(x, y, tri3, z, 50, )
plt.colorbar()
plt.title('Contour plot')
plt.xlabel('Longitude (degrees)')
plt.ylabel('Latitude (degrees)')
plt.show()

## grid plot

In [ ]:
plt.figure(figsize=(12,10))
plt.triplot(x,y,tri3,'go-', lw=.5, markersize=5)
plt.show()

## vector plot

In [ ]:
plt.figure(figsize=(12,10))
plt.gca().set_aspect('equal')
plt.quiver(x[::10], y[::10], U[0,::10], V[0,::10], angles='xy', scale_units='xy', scale=5)
plt.title('Vector plot')
plt.xlabel('Longitude (degrees)')
plt.ylabel('Latitude (degrees)')
plt.show()

## anim

In [ ]:
def anim(z,t,title=None):
    fig = plt.figure(figsize=(12,8))
    ax = fig.gca()
    ax.set_aspect('equal')
    v=np.linspace(z.min(),z.max(),10,endpoint=True)
    ims = []
    for i in tqdm(range(len(t))):
        im = ax.tricontourf(x, y, tri3, z[i,:], v, vmin=z.min(), vmax=z.max())
#        im = ax.contourf(x,y,z[i,:,:],v,vmin=v1,vmax=v2,latlon=True)
        add_arts = im.collections
        text = 'time={}'.format(t[i])
        #te = ax.text(90, 90, text)
        an = ax.annotate(text, xy=(0.05, 0.95), xycoords='axes fraction')
        ims.append(add_arts + [an])
    ax.set_title(title)    
#cbar_ax = fig.add_axes([0.05, 0.05, 0.85, 0.05])    
    cbar = fig.colorbar(im,ticks=v,orientation='vertical', extend='both')#,fraction=0.046, pad=0.04)
#plt.colorbar()
    return animation.ArtistAnimation(fig, ims, interval=200, blit=False,repeat=False)

In [ ]:
anim(h,t,title='Storm Surge')

In [ ]:
#anim(U,t,title='U - velocity')

### Mayavi Plot

In [ ]:
mlab.init_notebook()

In [ ]:
#convert lat/lon to x,y,z
w1 = np.cos(x) * np.cos(y)
w2 = np.cos(x) * np.sin(y)
w3 = np.sin(x)


In [ ]:
w1.shape, z.shape

In [ ]:
mlab.triangular_mesh(w1,w2,w3,tri3, scalars=z)